<a href="https://colab.research.google.com/github/Derevenitckaia/ML_itmo_course/blob/main/task_5_knn_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import io # чтобы считать данные из строки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

# Задача 1

Вам доступна таблица некоторых синтетических данных, на основании которых необходимо выполнить классификацию нового объекта, с помощью метода knn-ближайших соседей.

In [36]:
data_str = '''id,X,Y,Class
1,42,53,0
2,47,50,0
3,46,69,1
4,33,67,0
5,10,37,0
6,40,22,0
7,76,37,0
8,81,96,1
9,32,90,0
10,82,70,0
'''
df = pd.read_csv(io.StringIO(data_str), index_col = 'id')
df

,X,Y,Class
id,,,
1,42,53,0
2,47,50,0
3,46,69,1
4,33,67,0
5,10,37,0
6,40,22,0
7,76,37,0
8,81,96,1
9,32,90,0


In [37]:
def euclid_dist(a, b):
  n_a = len(a)
  n_b = len(b)
  dist = 0
  if n_a == n_b:
    for i in range(0, n_a):
      dist += (a[i] - b[i])**2
    return dist**0.5
  else: return(0)





In [38]:
new_obj = (67, 95)
df['euclid_dist'] = [euclid_dist(new_obj, row) for row_index, row in df[['X', 'Y']].iterrows()]
df.sort_values(by=['euclid_dist'])

,X,Y,Class,euclid_dist
id,,,,
8,81,96,1,14.035669
10,82,70,0,29.154759
3,46,69,1,33.421550
9,32,90,0,35.355339
4,33,67,0,44.045431
1,42,53,0,48.877398
2,47,50,0,49.244289
7,76,37,0,58.694122
6,40,22,0,77.833155


In [39]:
def manhattan_dist(a, b):
  n_a = len(a)
  n_b = len(b)
  dist = 0
  if n_a == n_b:
    for i in range(0, n_a):
      dist += np.abs((a[i] - b[i]))
    return dist
  else: return(0)


In [40]:
df['manhattan_dist'] = [manhattan_dist(new_obj, row) for row_index, row in df[['X', 'Y']].iterrows()]
df.sort_values(by=['manhattan_dist'])

,X,Y,Class,euclid_dist,manhattan_dist
id,,,,,
8,81,96,1,14.035669,15
9,32,90,0,35.355339,40
10,82,70,0,29.154759,40
3,46,69,1,33.421550,47
4,33,67,0,44.045431,62
2,47,50,0,49.244289,65
1,42,53,0,48.877398,67
7,76,37,0,58.694122,67
6,40,22,0,77.833155,100


# Задача 2
Первая таблица содержит данные о классификации писем на «спам»/«не спам» и общее количество слов, входящих в эти группы.
<br>Во второй таблице представлены данные, по уникальным словам, и числу их вхождений в указанные группы.
<br>Ваша задача построить **модель наивного байесовского классификатора** и определить класс, к которому будет
отнесено письмо, содержащее текст: '*Access Cash Remove Refund Million Investment Bonus'*

In [41]:
from sklearn.naive_bayes import GaussianNB

In [91]:
words_data = '''word,spam,ham
Unlimited,2,1
Million,4,2		
Online,0,0		
Offer,9,3		
Refund,4,7	
Remove,9,2		
Investment,29,5	
Membership,27,9
Bonus,11,5
Gift,6,10
'''

mail_data = '''name,spam,ham
mails,16,13
words,101,44
'''

In [ ]:
words_df = pd.read_csv(io.StringIO(words_data))
mail_df = pd.read_csv(io.StringIO(mail_data))
mail_df


In [132]:
def P(x, spam_or_ham, words_df, mail_df, r ):
  words_set = set(words_df['word'])
  if x in words_set:
    cnt_x_in_y = float(words_df.loc[words_df['word'] == x][spam_or_ham]) # количество слов X в классе y
  else: cnt_x_in_y = 0
  v = words_df.shape[0] #  количество слов в словаре
  cnt_words_in_y = float(mail_df.loc[mail_df['name'] == 'words'][spam_or_ham]) # количество слов в классе y
  return (1 + cnt_x_in_y)/(v + r + cnt_words_in_y)




In [142]:
test_message = 'Access Cash Remove Refund Million Investment Bonus'
test_message = test_message.split(' ')
r = 0
words_set = set(words_df['word'])
for word in test_message:
  if word not in words_set:
    r += 1
# print(r)
P_spam = float(mail_df.loc[mail_df['name'] =='mails']['spam'])/(float(mail_df.loc[mail_df['name'] =='mails']['spam']) + float(mail_df.loc[mail_df['name'] =='mails']['ham']))
print(P_spam)
P_ham = float(mail_df.loc[mail_df['name'] =='mails']['ham'])/(float(mail_df.loc[mail_df['name'] =='mails']['spam']) + float(mail_df.loc[mail_df['name'] =='mails']['ham']))
F_spam = np.log(P_spam)
F_ham = np.log(P_ham)
for word in test_message:
  # print(P(word, 'spam', words_df, mail_df, r))
  F_spam += np.log(P(word, 'spam', words_df, mail_df, r))
for word in test_message:
  F_ham += np.log(P(word, 'ham', words_df, mail_df, r))
  # print(P(word, 'ham', words_df, mail_df, r))
print(F_spam, F_ham)

0.5517241379310345
-22.278856889420677 -21.11962325019882


In [140]:
prob = 1/(1+np.exp(F_ham - F_spam))
prob

0.23880656479229995